In [11]:
import pandas as pd
from google.cloud import bigquery
import io

In [3]:
class Client:
    def __init__(self,SERVICE_ACCOUNT_JSON):
        self.SERVICE_ACCOUNT_JSON=SERVICE_ACCOUNT_JSON
        self.bigQuery_client=None
    def makeConnection(self):
        from google.cloud import bigquery
        self.bigQuery_client=bigquery.Client.from_service_account_json(self.SERVICE_ACCOUNT_JSON)
    def getClient(self):
        return self.bigQuery_client

In [6]:
SERVICE_ACCOUNT_JSON=r'D:\project\aerial-reef-332517-07a94b2ffe42.json'
client=Client(SERVICE_ACCOUNT_JSON)
client.makeConnection()
bigQueryClient=client.getClient()
print(bigQueryClient)

In [18]:
def create_dataset(dataset_name,location,description,bigQueryClient):
    try:
        dataset = bigquery.Dataset(dataset_name)
        dataset.location =location
        dataset = bigQueryClient.create_dataset(dataset, timeout=30)  # Make an API request.
        print("Created dataset {}.{}".format(bigQueryClient.project, dataset.dataset_id))
    except Exception as e:
        print("Error : {}".format(e))

In [17]:
dataset_name="aerial-reef-332517.Json_tables"
loaction="US"
description="Loading table iwth json data"
create_dataset(dataset_name,loaction,description,client.getClient())

error : 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/aerial-reef-332517/datasets?prettyPrint=false: Already Exists: Dataset aerial-reef-332517:Json_tables


Creating the table for nested json

In [20]:
project = bigQueryClient.project
dataset_ref = bigquery.DatasetReference(project, 'Json_tables')

schema = [
    bigquery.SchemaField("id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("first_name", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("last_name", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("dob", "DATE", mode="NULLABLE"),
    bigquery.SchemaField(
        "addresses",
        "RECORD",
        mode="REPEATED",
        fields=[
            bigquery.SchemaField("status", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("address", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("city", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("state", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("zip", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("numberOfYears", "STRING", mode="NULLABLE"),
        ],
    ),
]
table_ref = dataset_ref.table("my_table")
table = bigquery.Table(table_ref, schema=schema)
table = bigQueryClient.create_table(table)

print("Created table {}".format(table.full_table_id))

Created table aerial-reef-332517:Json_tables.my_table


In [34]:
def LoadTable(bigQueryClient,jsonfile,schema1,table_id):
    job_config = bigquery.LoadJobConfig(schema=schema1,
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,)
    uri = jsonfile

    load_job = bigQueryClient.load_table_from_uri(uri,table_id,location="US",job_config=job_config,)

    load_job.result()  # Waits for the job to complete.

    destination_table = bigQueryClient.get_table(table_id)
    print("Loaded {} rows.".format(destination_table.num_rows))

In [35]:
LoadTable(bigQueryClient,"gs://gcp_proj_temp/user.json",schema,table_ref)

Loaded 2 rows.
